In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer



In [2]:
data_path = "Dataset_analyse.csv"
df = pd.read_csv(data_path)

df = df.dropna()
df = df.drop(["title","total_spectator","hebdo_rank","rating_public","casting","visa","award","lang"], axis=1)
df

,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,copies,rating_press,budget
0,2007,sidney lumet,etatsunis,6960,thriller,8749.0,74208,60450.0,7,118,3.8,18000000.0
1,1999,jane campion,australie,6900,drame,10135.6,72806,59784.0,11,103,3.1,27422000.0
2,1997,tom shadyac,etatsunis,5160,comedie,64774.0,845332,422481.0,1,343,-1.0,45000000.0
3,2011,brett ratner,etatsunis,6240,comedie,9759.0,97963,76850.0,12,201,2.8,75000000.0
4,2013,jeanpaul salome,france,6300,comedie,12107.0,102817,72040.0,8,199,2.9,9260000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2380,2006,eric rochant,france,5820,comedie,17466.0,137261,112843.0,7,180,2.1,38520000.0
2381,2009,zack snyder,etatsunis,9780,comicbook,68152.0,377235,329087.0,2,537,3.7,100000000.0
2382,2000,mcg,etatsunis,5880,aventure action,91501.0,711821,616152.0,1,656,3.4,92000000.0
2384,2004,robert guediguian,france,6480,comedie,9616.0,67665,60122.8,14,117,3.3,383000000.0


In [3]:
X = df.drop(['first_week'], axis=1)
y = df.first_week

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore'), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

# param_grid = {'lasso__alpha': [15]}


# model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)
grid_score = model.score(X_train, y_train)

# best_alpha = model.best_params_['lasso__alpha']

/home/elie/Documents/Projets DEV IA/Brief_Popularity_Film/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 119248605161.50177, tolerance: 40225254354.265
  model = cd_fast.sparse_enet_coordinate_descent(


In [4]:
# y_pred_train = model.predict(X_train)


# residus =  y_pred_train - y_train


# weights = np.exp(-abs(residus) / residus.std())
# model.fit(X_train, y_train, lasso__sample_weight=weights)




y_pred = model.predict(X_test)

In [6]:

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
# print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle après suppression des valeurs extrêmes:
R2 Score: 0.8672
MAE: 115764.34
RMSE: 182122.13
